# Preprocessing CODE


In [1]:
import os
import numpy as np
import sys
from torch.utils.data import DataLoader

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.preprocessing.Derm7pt import *
from src import ImageConceptDataset
from src.preprocessing import *
from src.utils import get_paths, load_Derm_dataset

In [4]:
from src.config import PROJECT_ROOT

print(PROJECT_ROOT)

None


In [2]:
paths = get_paths()
dataset_handler = load_Derm_dataset(paths)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# Ensure text files exist
if not os.path.exists(paths['labels_file']):
    export_image_props_to_text(dataset_handler.df)

In [ ]:
verbose = True

In [ ]:
all_data = dataset_handler.df
all_data

In [ ]:
import numpy as np
import pandas as pd

all_labels = dataset_handler.get_labels(data_type='all', one_hot=True)
mapping_data = pd.read_csv(paths['mapping_file'], sep=' ', header=None, names=['img_id', 'img_path', 'img_type', 'case_id'])

concepts_matrix = []

for _, row in mapping_data.iterrows():
    case_id = row['case_id']
    instance_concepts = None
    for concept, vals in all_labels.items():
        if concept == 'DIAG':
            continue

        concept_one_hot = vals[case_id]

        if instance_concepts is None:
            instance_concepts = concept_one_hot
        else:
            # Concatenate horizontally (along axis 1)
            instance_concepts = np.hstack((instance_concepts, concept_one_hot))

    concepts_matrix.append(instance_concepts)

concepts_matrix1 = np.array(concepts_matrix)

In [ ]:
# Get labels and concepts
image_labels = one_hot_encode_labels(paths['labels_file'], paths['classes_path'], verbose=verbose)
concepts_matrix = encode_image_concepts(dataset_handler, paths['mapping_file'], verbose=verbose)

# Load and transform images
image_tensors, image_paths = load_and_transform_images(paths['dir_images'], paths['mapping_file'], resol=299, use_training_transforms=True, batch_size=32, verbose=verbose)

In [ ]:
# Filter if needed
if image_labels.shape[0] != len(image_tensors):
    filtered_image_labels, filtered_concepts_matrix = filter_concepts_labels(
        paths['mapping_file'], image_tensors, image_paths, image_labels, concepts_matrix
    )
else:
    filtered_image_labels, filtered_concepts_matrix = image_labels, concepts_matrix

if verbose:
    print("Labels shape:", filtered_image_labels.shape)
    print("Concepts shape:", filtered_concepts_matrix.shape)
    print("Image tensors length:", len(image_tensors))


In [ ]:
tensors_dict, concepts_dict, labels_dict = split_data_by_indices(
    image_tensors, image_paths, filtered_concepts_matrix, filtered_image_labels,
    paths, verbose=verbose
)

train_concept_labels = concepts_dict['train']
val_concept_labels = concepts_dict['val']
test_concept_labels = concepts_dict['test']

train_img_labels = labels_dict['train']
val_img_labels = labels_dict['val']
test_img_labels = labels_dict['test']

train_tensors = tensors_dict['train']
val_tensors = tensors_dict['val']
test_tensors = tensors_dict['test']


In [ ]:
# concept processing
from src.config import DERM7PT_CONFIG

class_level_concepts = compute_class_level_concepts(train_concept_labels, None, train_img_labels)

# apply class-level concepts to each instance
if True:
    train_concept_labels, val_concept_labels, test_concept_labels = apply_class_concepts_to_instances(
        class_level_concepts, DERM7PT_CONFIG, train_img_labels, train_concept_labels,
        test_img_labels, test_concept_labels, val_img_labels, val_concept_labels)

common_concept_indices = select_common_concepts(class_level_concepts, min_class_count=0, CUB=False)
train_concept_labels = train_concept_labels[:, common_concept_indices]
val_concept_labels = val_concept_labels[:, common_concept_indices]
test_concept_labels = test_concept_labels[:, common_concept_indices]

In [ ]:
import numpy as np
from src.config import PROJECT_ROOT

np.save(os.path.join(PROJECT_ROOT, 'output', 'Derm7pt', 'class_level_concepts.npy'), class_level_concepts)

In [ ]:
common_concept_indices.shape

In [ ]:
# CREATE TRAIN AND TEST DATASET
train_dataset = ImageConceptDataset(
    image_tensors=train_tensors,
    concept_labels=train_concept_labels,
    image_labels=train_img_labels
)

test_dataset = ImageConceptDataset(
    image_tensors=test_tensors,
    concept_labels=test_concept_labels,
    image_labels=test_img_labels
)


In [ ]:
# CREATE DATALOADERS FROM DATASETS
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, drop_last=False)
